# System Setting

## My System info
### System Spec
- NVIDIA Geforce RTX2070 Super
- Ryzen 3700X
- Windows 10

### Software info
- Visual Studio Code  
```bash
Version: 1.56.0 (user setup)  
Commit: cfa2e218100323074ac1948c885448fdf4de2a7f  
Date: 2021-05-04T22:09:06.405Z  
Electron: 12.0.4  
Chrome: 89.0.4389.114  
Node.js: 14.16.0  
V8: 8.9.255.24-electron.0  
OS: Windows_NT x64 10.0.19041  
```


- python version
```bash
c:\Program' 'Files\Python38\Python.exe --version
Python 3.8.5
```

- pip3 version
```bash
c:\Program' 'Files\Python38\Scripts\pip3.exe --version
pip 21.1
```
- CUDA Version
```bash
nvcc --version
```

```
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Mar_21_19:24:09_Pacific_Daylight_Time_2021
Cuda compilation tools, release 11.3, V11.3.58
Build cuda_11.3.r11.3/compiler.29745058_0
```

- Download torchtext
```bash
c:\Program' 'Files\Python38\Scripts\pip3.exe install -U torchtext
```

- Download torch on CUDA11.1, venv
```bash
c:\Program' 'Files\Python38\Scripts\pip3.exe install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
```

- Download spaCy(English, German) on CUDA11.2
```bash
c:\Program' 'Files\Python38\Scripts\pip3.exe install -U pip setuptools wheel
c:\Program' 'Files\Python38\Scripts\pip3.exe install -U spacy[cuda112]
c:\Program' 'Files\Python38\Python.exe -m spacy download en_core_web_sm
c:\Program' 'Files\Python38\Python.exe -m spacy download de_core_news_sm
```

In [5]:
import torch
import torch.nn as nn #basic buildling block for graphs
import torch.optim as optim #optimization algorithms > such as SGD, Adam

#https://github.com/pytorch/text/tree/master/torchtext/legacy
from torchtext.legacy.datasets import Multi30k 
from torchtext.legacy.data import Field, BucketIterator

import spacy
spacy.prefer_gpu() #I prefer GPU
import numpy as np

import random
import math
import time

In [6]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
#import spacy.cli
spacy.cli.download('de_core_news_sm')
spacy.cli.download('en_core_web_sm')

In [9]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [10]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1] 
    #순서 반전. 처리에 용이하므로 의도적으로 반전
    #[Start:End:Step]인데, Start와 End를 비우고 Step을 -1로 하면 순서가 반전된다.

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [11]:
SRC = Field(tokenize = tokenize_de, #Source Data
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True) #lower case

TRG = Field(tokenize = tokenize_en, #Target Data
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True) #lower case

In [12]:
#Training
#It takes quite a long time
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), #specify which languages to use as a source(first) and target(last)
                                                    fields = (SRC, TRG)) #which fields to use for the source(first) and target(last)

downloading training.tar.gz
c:\GitHub\LearningProgrammingLanguages\PyTorch\.data\multi30k\training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 607kB/s] 
downloading validation.tar.gz
c:\GitHub\LearningProgrammingLanguages\PyTorch\.data\multi30k\validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 197kB/s]
downloading mmt_task1_test2016.tar.gz
c:\GitHub\LearningProgrammingLanguages\PyTorch\.data\multi30k\mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 187kB/s]


In [13]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [14]:
print(vars(train_data.examples[0])) #멤버변수에 대한 내용을 보여줌. 
#{변수명:값}으로 dictionary를 만들어서 보여줌
#매칭은 안되고 번역한 부분만 보여주는듯?
#점(period)을 통해 독일어가 잘 반전되어 저장되었음을 확인 가능

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [15]:
#vocabulary 생성하기(vocab 멤버): 유일한 단어에 정수 인덱스를 부여함.
#min_freq 옵션으로 최소한 몇번 이상 나타난 단어만 저장할 것인지 결정.
#여기서는 최소 2번 이상 등장한 단어만 단어장에 저장하도록 한다. > 학습 효과를 높이기 위함
#원본(?)에서 단 한번만 등장한 토큰은 <unk>(unknown)으로 변환된다.
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [16]:
#vocab 멤버로 고유 토큰을 찾을 수 있음
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7853
Unique tokens in target (en) vocabulary: 5893


In [17]:
#if GPU is detected, put the tensors on the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
#batch 데이터들은 source의 길이와 target의 길이를 맞춰야 하는데,
#torchText가 이를 자동으로 처리해준다.
#define batch size and iterators
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

print(len(train_iterator), len(valid_iterator), len(test_iterator), sep=' ')

227 8 8


In [19]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [20]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [21]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [22]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [23]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,898,501 trainable parameters


In [25]:
optimizer = optim.Adam(model.parameters())

In [26]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [27]:

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [28]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [29]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [30]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 32s
	Train Loss: 5.051 | Train PPL: 156.163
	 Val. Loss: 4.980 |  Val. PPL: 145.545
Epoch: 02 | Time: 0m 32s
	Train Loss: 4.500 | Train PPL:  90.017
	 Val. Loss: 4.891 |  Val. PPL: 133.054
Epoch: 03 | Time: 0m 31s
	Train Loss: 4.209 | Train PPL:  67.257
	 Val. Loss: 4.629 |  Val. PPL: 102.424
Epoch: 04 | Time: 0m 30s
	Train Loss: 4.005 | Train PPL:  54.887
	 Val. Loss: 4.512 |  Val. PPL:  91.059
Epoch: 05 | Time: 0m 29s
	Train Loss: 3.842 | Train PPL:  46.598
	 Val. Loss: 4.431 |  Val. PPL:  83.996
Epoch: 06 | Time: 0m 31s
	Train Loss: 3.708 | Train PPL:  40.756
	 Val. Loss: 4.264 |  Val. PPL:  71.126
Epoch: 07 | Time: 0m 31s
	Train Loss: 3.582 | Train PPL:  35.958
	 Val. Loss: 4.148 |  Val. PPL:  63.289
Epoch: 08 | Time: 0m 31s
	Train Loss: 3.440 | Train PPL:  31.186
	 Val. Loss: 4.123 |  Val. PPL:  61.736
Epoch: 09 | Time: 0m 31s
	Train Loss: 3.295 | Train PPL:  26.978
	 Val. Loss: 3.977 |  Val. PPL:  53.368
Epoch: 10 | Time: 0m 31s
	Train Loss: 3.183 | Train PPL

In [31]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.963 | Test PPL:  52.621 |
